## importing Libraries 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img

2024-05-17 15:12:19.101814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 15:12:19.101938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 15:12:19.246133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import shutil

## importing our dataset (ERG) 

In [3]:
# Read the Excel file into a Pandas DataFrame
SR = pd.read_excel("/kaggle/input/electroretinogram/01 Appendix 1.xlsx", sheet_name='Scotopic 2.0 ERG Response')
PR = pd.read_excel("/kaggle/input/electroretinogram/01 Appendix 1.xlsx", sheet_name='Photopic 2.0 ERG Response')
MR = pd.read_excel("/kaggle/input/electroretinogram/01 Appendix 1.xlsx", sheet_name='Maximum 2.0 ERG Response')


/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [30]:
MR.drop(columns = MR.columns[124:], inplace=True)
PR.drop(columns = PR.columns[110:], inplace=True)
SR.drop(columns = SR.columns[80:], inplace=True)

In [31]:
mapping = {'The functional activity of the central and peripheral parts of the retina in both eyes is preserved.':'abnormal','Pupil is narrow.':'abnormal','The functional activity of the central and peripheral parts of the retina in both eyes is preserved.':'abnormal','Registration of ERG with a narrow pupil.':'abnormal','Registration of the ERG of a narrow pupil from the skin of the eyelid.':'abnormal','ERG pupil is narrow.':'abnormal','Registration of ERG from the skin of the eyelid, the pupil is narrow.':'abnormal','Registration of ERG with a narrow pupil from the skin of the eyelid.':'abnormal','The functional activity of the central and peripheral parts of the retina in both eyes is preserved.':'abnormal','High level of interference due to increased physical activity of the child.':'abnormal','The functional activity of the retina is preserved.':'normal','The functional activity of the retina at the OU is preserved.': 'normal','The functional activity of the retina is preserved.':'normal','No negative dynamics compared to previous data.': 'normal','The functional activity of the retina is preserved.': 'normal','The functional activity of the central and peripheral parts of the retina in both eyes is preserved.':'normal','No diagnosis.': 'normal','The signal is within normal limits.': 'normal','The functional activity of the retina is preserved, corresponds to the age norm symmetrically on the OU.': 'normal','The functional activity of the retina in both eyes is preserved.': 'normal','The amplitude-time characteristics of the EP in a homogeneous field correspond to the norm.': 'normal','The functional activity of the retina in both eyes is preserved. High level of interference due to increased motor activity of the child.': 'normal','The functional activity of the retina at the OU is preserved. No macular pathology was revealed.': 'The functional activity of the central and peripheral parts of the retina in both eyes is preserved.','No diagnosis.': 'normal','Electrogenesis of the central parts of the retina normally does not exclude dystrophy of the rod apparatus of the retina with a favorable prognosis of the course.': 'normal','The functional activity of the central and peripheral parts of the retina in both eyes is preserved.':'normal','Correct configuration potentials. The amplitude-time characteristics of the a- and b-waves of the rod maximal and cone responses correspond to the norm.': 'normal', 'Perhaps the reason for the violation of twilight vision is hypovitaminosis.': 'normal' ,'Hereditary cone dystrophy.': 'abnormal', 
          'Retinal rod dystrophy with a favorable prognosis is not excluded.': 'abnormal','Decreased the amplitude of the b-wave of the maximum and rod responses.': 'abnormal','Electrogenesis of the peripheral parts of the retina in the norm, the electrogenesis of the central parts of the retina is sharply impaired. Perhaps there is hereditary cone dystrophy.': 'abnormal','Reduced functional activity of the retina changes at the level of the outer and middle layers in the central and peripheral parts.': 'abnormal','Combination of congenital myopia and congenital hemeralopia.': 'abnormal','OD - outside and in the upper section of the old chorioretinal focus.': 'abnormal','Rod dystrophy of the retina with a favorable prognosis.': 'abnormal','The functional activity of the peripheral parts of the retina is normal, the functional activity of the central parts of the retina is moderately reduced.': 'abnormal','Hereditary rod dystrophy of the retina is possible.': 'abnormal','Central dystrophy with cone-rod dysfunction is possible.': 'abnormal','The preservation of the retinal electrogenesis from its central and peripheral parts, as well as the preservation of conductivity along the pathways of the visual analyzer, can be expected to improve visual functions after restoration of the transparency of the OS optical media.': 'abnormal','Retinal cone-rod dystrophy.': 'abnormal', 'Pronounced organic changes in the outer and inner layers in the center and periphery of the retina.': 'abnormal', 'Moderate pronounced change in the outer and middle layers of the central and peripheral parts of the retina of the right eye.': 'abnormal', 'Decrease in electrogenesis and functional activity of the retina.': 'abnormal', 'Retinal rod dystrophy with a favorable prognosis is possible.': 'abnormal', 'Signs of moderate disturbances in the electrogenesis of the central parts of the retina in the left eye.': 'abnormal', 'OU - maximum ERG b-wave reduction. Central dystrophy with cone-rod dysfunction is possible.': 'abnormal', 'The functional activity of the retina of the left eye is protected by a moderate decrease in the functional activity of the retina of the right eye.': 'abnormal', 'The functional activity of the retina OD is preserved on the OS and is moderately reduced (changes at the level of the outer and middle layers of the retina in the central and peripheral regions).': 'abnormal', 'The functional activity of the central parts of the retina is moderately reduced.': 'abnormal', 'Changes in the bioelectrical activity of the retina against the background of high myopia.': 'abnormal', 'Hereditary rod dystrophy of the retina is not excluded.': 'abnormal', 'Decrease of the B-wave amplitude by 3 times on OD, 2 times on OS.': 'abnormal', 'OU - a pronounced decrease in the b-wave of the maximum response is reduced.': 'abnormal'}
MR.replace(mapping, inplace=True)
PR.replace(mapping, inplace=True)
SR.replace(mapping, inplace=True)

In [22]:
patiant_Diagnosis_M = MR.iloc[4 , :122].tolist()
patiant_Diagnosis_P = PR.iloc[4 , :106].tolist()
patiant_Diagnosis_S = SR.iloc[4 , :74].tolist()

In [23]:
patiant_id_M = MR.iloc[3 , :122].tolist()
patiant_id_P = PR.iloc[3 , :106].tolist() #20 
patiant_id_S = SR.iloc[3 , :74].tolist()

In [24]:
M_R =['MR']*122
P_R =['PR']*106
S_R =['SR']*74

In [25]:
len(patiant_Diagnosis_M+patiant_Diagnosis_P+patiant_Diagnosis_S)
len(patiant_id_P)

106

In [32]:
# Create a dictionary from the two lists
dict = {'Patient_id': patiant_id_M+patiant_id_P+patiant_id_S, 'Label': patiant_Diagnosis_M+patiant_Diagnosis_P+patiant_Diagnosis_S ,'signal_type':M_R+P_R+S_R}

# Create a new dataframe from the dictionary
df = pd.DataFrame(dict)
#df=df.sample(frac=1).reset_index(drop=True)

In [33]:
filtered_df = df[df['Label'] == 'Diagnosis']

In [34]:
df = df[~(df['Label'] == 'Diagnosis')] #the negation operator ~ in front of the boolean mask, we invert the mask to select the rows that do not contain the specific value.
df=df.reset_index(drop=True)
df.groupby(['Label']).count()

,Patient_id,signal_type
Label,,
abnormal,107,107
normal,192,192


In [13]:
!mkdir signals
!mkdir augmented_images

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [14]:
def repeat_val(x):
    if x<200 : k=4 
    elif x>200 and x<=400 : k=3 
    elif x>400 and x<=550 : k=2 
    return k  

## Transform the signal by Ricker wavelet (mexican hat)

In [15]:
signal_PR=[]
signal_PR_id=[]
path='/kaggle/working/signals/'
for j in range(0,2):
    for i in range (1,106):
        signal = np.array(PR.iloc[21: ,i].tolist())
        signal_values = signal[np.logical_not(np.isnan(signal))]
        k = repeat_val(len(signal_values))

        signal_values = np.tile(signal_values.tolist()+[0]*30,k-j)
        
        coefficients, frequencies = pywt.cwt(signal_values, scales=np.arange(1, 128), wavelet='mexh')
        plt.figure(figsize=(5.12, 5.12))
        plt.imshow(abs(coefficients) ** 2, cmap='gray', aspect='auto',vmax=abs(coefficients).max(), vmin=-abs(coefficients).min())
        plt.axis('off')
        plt.savefig(os.path.join(path, f'PR_{i}_{j}_f.png'))
        plt.close()
        signal_PR.append(path + f'PR_{i}_{j}_f.png')
        signal_PR_id.append(f'PR_{i}_{j}_f')
    

In [16]:
signal_MR=[]
signal_MR_id=[]
for j in range(0,2):
    for i in range (1,122):
        signal = np.array(MR.iloc[22: ,i].tolist())  
        signal_values = signal[np.logical_not(np.isnan(signal))]
        k = repeat_val(len(signal_values))

        signal_values = np.tile(signal_values.tolist()+[0]*30,k-j)
        
        coefficients, frequencies = pywt.cwt(signal_values, scales=np.arange(1, 128), wavelet='mexh')
        plt.figure(figsize=(5.12, 5.12))
        plt.imshow(abs(coefficients) ** 2, cmap='gray', aspect='auto',vmax=abs(coefficients).max(), vmin=-abs(coefficients).min())
        plt.axis('off')
        plt.savefig(os.path.join(path, f'MR_{i}_{j}_f.png'))
        plt.close()
        signal_MR.append(path + f'MR_{i}_{j}_f.png')
        signal_MR_id.append(f'MR_{i}_{j}_f')

In [17]:
signal_SR=[]
signal_SR_id=[]
for j in range(0,2):
    for i in range (1,74):
        signal = np.array(SR.iloc[21: ,i].tolist())  
        signal_values = signal[np.logical_not(np.isnan(signal))]
        k = repeat_val(len(signal_values))

        signal_values = np.tile(signal_values.tolist()+[0]*0,k-j)
          
        coefficients, frequencies = pywt.cwt(signal_values, scales=np.arange(1, 128), wavelet='mexh')
        plt.figure(figsize=(5.12, 5.12))
        plt.imshow(abs(coefficients) ** 2, cmap='gray', aspect='auto',vmax=abs(coefficients).max(), vmin=-abs(coefficients).min())
        plt.axis('off')
        plt.savefig(os.path.join(path, f'SR_{i}_{j}_f.png'))
        plt.close()
        signal_SR.append(path + f'SR_{i}_{j}_f.png')
        signal_SR_id.append(f'SR_{i}_{j}_f')


## the final data frame

In [37]:
df = pd.DataFrame({'Label': np.tile(df['Label'],2)})
df['img_Path'] = signal_MR+signal_PR+signal_SR
df['img_id'] = signal_MR_id+signal_PR_id+signal_SR_id
df.groupby(['Label']).count()

,img_Path,img_id
Label,,
abnormal,214,214
normal,384,384


In [38]:
df.to_csv('data1.csv')

In [39]:
mapping = {'normal': int(0), 'abnormal': int(1)}  
df.replace(mapping, inplace=True)
df = df.reset_index(drop=True)

/tmp/ipykernel_33/588916444.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(mapping, inplace=True)


In [40]:
droped_normal = df.query('Label == 0').sample(n=150)
df.drop(droped_normal.index, inplace=True)
df=df.reset_index(drop=True)

In [41]:
df.groupby(['Label']).count()

,img_Path,img_id
Label,,
0,234,234
1,214,214


In [42]:
df.to_csv('Scalogram_grayscale_images.csv')
droped_normal.to_csv('test_normal.csv')

In [43]:
augmented_folder_path='/kaggle/working/augmented_images/'
lables_aug=[]
paths_aug=[]
id_aug=[]
# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2)
)

labels = ['rotated','sheered', 'zoomed', 'horizontal_flipped' ,'brightness_changed']

for n in range (len(df.img_Path)):
    # Loading a sample image 
    img = load_img(df.img_Path[n]) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape)
    j=0
    # Loop through the 5 augmented images and save each image with its corresponding label
    for i, batch in enumerate(datagen.flow(x, batch_size=1,save_format='png')):
        # Get the augmented image array
        augmented_image = batch[0]
        # Get the corresponding label
        label = labels[j]
        j+=1
        # Save the augmented image with its corresponding label
        img = array_to_img(augmented_image)
        img.save(os.path.join(augmented_folder_path, f'{df.img_id[n]}_{label}.png'))
        paths_aug.append(f'{augmented_folder_path}/{df.img_id[n]}_{label}.png')  
        lables_aug.append(df.Label[n])
        id_aug.append(f'{df.img_id[n]}_{label}')
        if j==5:
            break  

In [47]:
# Create a dictionary from the two lists
dict = {'img_Path': paths_aug+list(df.img_Path), 'Label': lables_aug+list(df.Label), 'img_id': id_aug+list(df.img_id) }

# Create a new dataframe from the dictionary
final = pd.DataFrame(dict)
#final=final.sample(frac=1).reset_index(drop=True)

,img_Path,Label,img_id
0,/kaggle/working/augmented_images//MR_1_0_f_rot...,0,MR_1_0_f_rotated
1,/kaggle/working/augmented_images//MR_1_0_f_she...,0,MR_1_0_f_sheered
2,/kaggle/working/augmented_images//MR_1_0_f_zoo...,0,MR_1_0_f_zoomed
3,/kaggle/working/augmented_images//MR_1_0_f_hor...,0,MR_1_0_f_horizontal_flipped
4,/kaggle/working/augmented_images//MR_1_0_f_bri...,0,MR_1_0_f_brightness_changed
...,...,...,...
2683,/kaggle/working/signals/SR_66_1_f.png,0,SR_66_1_f
2684,/kaggle/working/signals/SR_68_1_f.png,0,SR_68_1_f
2685,/kaggle/working/signals/SR_69_1_f.png,0,SR_69_1_f
2686,/kaggle/working/signals/SR_72_1_f.png,1,SR_72_1_f


In [45]:
final.to_csv('Scalogram_grayscale_augumented_images.csv')

In [46]:
shutil.make_archive('/kaggle/working/output', 'zip', '/kaggle/working/')

'/kaggle/working/output.zip'